<div align="right">  
    Grupo 13  
    
    André Neves da Costa - A95869 
    Filipe José Silva Castro - A96156 
</div>

# Algoritmo BubbleSort

O programa Python seguinte implementa o algoritmo de bubble sort para ordenação in situ de um array de inteiros seq:

```python

seq = [-2,1,2,-1,4,-4,-3,3]
changed = True
while changed:
    changed = False
    for i in range(len(seq) - 1):
        if seq[i] > seq[i+1]:
            seq[i], seq[i+1] = seq[i+1], seq[i]
            changed = True
pass
```

# Esquecificação do algoritmo

O algortimo apresentado anteriormente pode ser descrito pelas seguintes pré e pós condições:

**pré-condição:** $len(seq) > 1 \wedge changed = True$

**pós-condição:** $\forall_{0 \le i < len(seq)-1}seq[i] < seq[i+1]$

portanto temos:

```python
assume len(seq) > 1 and changed = True
seq = [-2,1,2,-1,4,-4,-3,3]
changed = True
while changed:
    changed = False
0:  for i in range(len(seq) - 1):
1:      if seq[i] > seq[i+1]:
2:          seq[i], seq[i+1] = seq[i+1], seq[i]
3:          changed = True
4:    pass
assert forall i . 0 <= i > len(seq) - 1 -> seq[i] < seq[i+1]
```

Agora, queremos modelar a atribuição $\,\mathtt{seq}\gets \mathit{exp}(\mathtt{seq})\,$ construindo uma relação de transição $trans(seq,seq′)$ a partir do ciclo for presente no algoritmo, e com isso temos:

O estado inicial é caracterizado pelo seguinte predicado:

$$pc = 0 \wedge changed = False \wedge i = 0$$

As transições de estados são caracterizadas pelos seguintes predicados:

$$pc = 0 \wedge pc' = 1 \wedge i < len(seq) - 1$$
$$\vee$$
$$pc = 1 \wedge pc' = 2 \wedge seq[i] > seq[i + 1]$$
$$\vee$$
$$pc = 2 \wedge pc' = 3 \wedge seq[i]' = seq[i + 1] \wedge seq[i + 1]' = seq[i]$$
$$\vee$$
$$pc = 3 \wedge pc' = 0 \wedge changed' = True \wedge i' = i + 1$$
$$\vee$$
$$pc = 0 \wedge pc' = 4 \wedge i \ge len(seq) - 1$$
$$\vee$$
$$pc = 1 \wedge pc' = 0 \wedge seq[i] \le seq[i + 1]$$

# Implementação em Python

Para esta parte do trabalho teriamos então de definir um `declare`, `init` e `trans`, e estando a trabalhar com uma lista experimentamos defini-la como vários inteiros o que acabou por não resultar, tentamos então usar o ArrayType, nao chegou a resultar também, porque quando chamamos de alguma maneira $seq[i]$ ou $seq[i+1]$, como i é uma variável contínua, temos um erro, pois i ainda nao estaria definida então não seria possível usá-la para recolher os valores de seq

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import *

seq = [-2,1,2,-1,4,-4,-3,3]

def declare(i):
    state={}
    state['pc'] = Symbol('pc'+'_'+str(i),INT)
    state['changed'] = Symbol('changed'+'_'+str(i),INT)
    state['i'] = Symbol('i'+'_'+str(i),INT)
    state['list']=Symbol('_'+str(i),ArrayType(INT, INT))
    return state

def init(state):
    a = Symbol("a", ArrayType(INT, INT))
    for n in range(len(seq)):
        a = (Store(a, Int(n), Int(seq[n])))
    return And(Equals(state['pc'],Int(0)),Equals(state['changed'],Int(0)),Equals(state['i'],Int(0)),Equals(state['list'],a))


def trans(curr,prox):
    t01 = And(Equals(curr['pc'], Int(0)),Equals(prox['pc'],Int(1)),LT(curr['i'],Int(len(seq) - 1)))
    t12 = And(Equals(curr['pc'], Int(1)),Equals(prox['pc'],Int(2)),GT(simplify(curr['list']).arg(curr['i']),simplify(curr['list']).arg(curr['i'] + Int(1))))
    t23 = And(Equals(curr['pc'], Int(2)),Equals(prox['pc'],Int(3)),Store(prox['list'],curr['i'],simplify(curr['list']).arg(curr['i'] + Int(1))),Store(prox['list'],curr['i'] + Int(1),simplify(curr['list']).arg(curr['i'])))
    t30 = And(Equals(curr['pc'], Int(3)),Equals(prox['pc'],Int(0)),Equals(prox['changed'],Int(1)),Equals(prox['i'],curr['i'] + Int(1)))
    t04 = And(Equals(curr['pc'], Int(0)),Equals(prox['pc'],Int(4)),GE(curr['i'],Int(len(seq)-1)))
    t10 = And(Equals(curr['pc'], Int(1)),Equals(prox['pc'],Int(0)),LE(simplify(curr['list']).arg(curr['i']),simplify(curr['list']).arg(curr['i'] + Int(1))))
    
    return Or(t01,t12,t23,t30,t04,t10)

In [ ]:
def gera_traco(declare,init,trans,k):
    with Solver(name="z3") as s:
        
        trace = [declare(i) for i in range(k)]
        
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i],trace[i+1]))
            
        if s.solve():
            for i in range(k):
                print()
                print("State:",i)
                print("pc =", float(s.get_py_value(trace[i]['v'])))
                print("changed =", float(s.get_py_value(trace[i]['V'])))
                print("i =", float(s.get_py_value(trace[i]['t'])))
                print("[")
                for n in len(seq):
                    print(float(s.get_py_value(trace[i]['seq[' + str(n) + ']'])))
                print("]")
                
        else:
            print("Não foi possível resolver")
        
gera_traco(declare,init,trans,12)